# Kerr equatorial eccentric fluxes

## Define location of data directory

In [88]:
data_dir = "/Users/znasipak/Library/CloudStorage/OneDrive-UniversityofSouthampton/FluxData/data/"

## Define coordinates for interpolated grids

In [68]:
import numpy as np
import pandas as pd
import os
from multispline.spline import CubicSpline, TricubicSpline # pip install multispline

### Define boundaries of the grid

In [69]:
CBUFFERVALUE = 0.05
PCRITVALUE = 10
PMAXVALUE = 30
AMAXVALUE = 0.99
AMINVALUE = 0.
EMAXVALUE = 0.8

In [89]:
spl_db = pd.read_table(data_dir+"RegionI_Coeffs.txt")

In [71]:
Bspl = CubicSpline(spl_db["a"].to_numpy(), spl_db["B"].to_numpy())
Mspl = CubicSpline(spl_db["a"].to_numpy(), spl_db["M"].to_numpy())

In [72]:
def get_boundaryI_coeffs(a):
    return Mspl(a), Bspl(a) + CBUFFERVALUE

def get_boundaryII_coeffs(a):
    return 0.004466205356102843, 0.3533794643897157

def minimum_simulated_separation(a, e):
    mcoeff, bcoeff = get_boundaryI_coeffs(a)
    return mcoeff * e + bcoeff

def maximum_simulated_separation(a, e):
    return PMAXVALUE

def critical_simulated_separation(a, e):
    return PCRITVALUE

def maximum_simulated_eccentricity(a, p):
    if p < PCRITVALUE:
        acoeff, ccoeff = get_boundaryII_coeffs(a)
        return acoeff*p**2 + ccoeff
    else:
        return EMAXVALUE

def minimum_simulated_eccentricity(a, p):
    return 0

def maximum_simulated_spin(p, e):
    return AMAXVALUE

def minimum_simulated_spin(p, e):
    return AMINVALUE

### Region A Transformations

In [73]:
def transform_ape_to_xyz_regionA(a, p, e):
    pmin = minimum_simulated_separation(a, e)
    pmax = PCRITVALUE
    x = (p - pmin) / (pmax - pmin)

    emax = maximum_simulated_eccentricity(a, p)
    y = e/emax

    amax = AMAXVALUE
    z = a/amax

    return x, y, z

def transform_xyz_to_ape_regionA(x, y, z):
    amax = AMAXVALUE
    a = z*amax
    mcoeff, bcoeff = get_boundaryI_coeffs(a)
    acoeff, ccoeff = get_boundaryII_coeffs(a)

    pmax = PCRITVALUE
    alpha = 1 - x
    beta = x*pmax
    delta = y*mcoeff

    A = alpha*delta*acoeff
    B = -1
    C = beta + alpha*(delta*ccoeff + bcoeff)
    if A == 0:
        p = -C/B
    else:
        p = (-B - np.sqrt(B**2 - 4*A*C))/(2*A)

    emax = maximum_simulated_eccentricity(a, p)
    e = y*emax

    return a, p, e

def transform_xyz_to_xyzbar_regionA(x, y, z):
    xbar = 2*(1 - (1/(1 + np.sqrt(np.abs(x)))))
    ybar = y
    zbar = (1 - z)**(1/3)
    return xbar, ybar, zbar

def transform_xyzbar_to_xyz_regionA(xbar, ybar, zbar):
    x = (2/(2 - xbar) - 1)**2
    y = ybar
    z = 1 - zbar**3
    return x, y, z

def transform_ape_to_xyzbar_regionA(a, p, e):
    x, y, z = transform_ape_to_xyz_regionA(a, p, e)
    return transform_xyz_to_xyzbar_regionA(x, y, z)

def transform_xyzbar_to_ape_regionA(xbar, ybar, zbar):
    x, y, z = transform_xyzbar_to_xyz_regionA(xbar, ybar, zbar)
    return transform_xyz_to_ape_regionA(x, y, z)

### Region B Transformations

In [74]:
def transform_ape_to_xyz_regionB(a, p, e):
    pmin = PCRITVALUE
    pmax = PMAXVALUE
    x = (p - pmin) / (pmax - pmin)

    emax = EMAXVALUE
    y = e/emax

    amax = AMAXVALUE
    z = a/amax

    return x, y, z

def transform_xyz_to_ape_regionB(x, y, z):
    amax = AMAXVALUE
    a = z*amax

    pmin = PCRITVALUE
    pmax = PMAXVALUE
    p = x*(pmax - pmin) + pmin

    emax = EMAXVALUE
    e = y*emax

    return a, p, e

def transform_xyz_to_xyzbar_regionB(x, y, z):
    xbar = x
    ybar = y
    zbar = (1 - z)**(1/3)
    return xbar, ybar, zbar

def transform_xyzbar_to_xyz_regionB(xbar, ybar, zbar):
    x = xbar
    y = ybar
    z = 1 - zbar**3
    return x, y, z

def transform_ape_to_xyzbar_regionB(a, p, e):
    x, y, z = transform_ape_to_xyz_regionB(a, p, e)
    return transform_xyz_to_xyzbar_regionB(x, y, z)

def transform_xyzbar_to_ape_regionB(xbar, ybar, zbar):
    x, y, z = transform_xyzbar_to_xyz_regionB(xbar, ybar, zbar)
    return transform_xyz_to_ape_regionB(x, y, z)

In [75]:
def valid_parameters_test_regionA(a, p, e):
    emax = maximum_simulated_eccentricity(a, p)
    pmin = minimum_simulated_separation(a, e)
    emin = 0
    amin = AMINVALUE
    amax = AMAXVALUE
    if e < emin or e > emax or p < pmin or a < amin or a > amax:
        return False
    else:
        return True
    
def valid_parameters_test_regionB(a, p, e):
    emax = EMAXVALUE
    pmax = PMAXVALUE
    emin = 0
    amin = AMINVALUE
    amax = AMAXVALUE
    if e < emin or e > emax or p > pmax or a < amin or a > amax:
        return False
    else:
        return True
    
def valid_parameters_test(a, p, e):
    if p < PCRITVALUE:
        return valid_parameters_test_regionA(a, p, e)
    else:
        return valid_parameters_test_regionB(a, p, e)

## Load flux data

In [90]:
dataA = pd.read_table(data_dir+"RegA_3DGrid_NX257NY129NZ129_C005_output.txt")
dataB = pd.read_table(data_dir+"RegB_3DGrid_NX257NY129NZ129_C005_output.txt")

In [78]:
NX = 257
NY = 129
NZ = 129
xsamples = np.linspace(0, 1, NX)
ysamples = np.linspace(0, 1, NY)
zsamples = np.linspace(0, 1, NZ)

In [79]:
def energy_flux_pn(p):
    return p**(-5.)

def momentum_flux_pn(p):
    return p**(-3.5)

In [80]:
ENormA = energy_flux_pn(dataA["p0"].to_numpy())
ENormB = energy_flux_pn(dataB["p0"].to_numpy())

LNormA = momentum_flux_pn(dataA["p0"].to_numpy())
LNormB = momentum_flux_pn(dataB["p0"].to_numpy())

In [81]:
EdotDataA = dataA["Edot"].to_numpy()/ENormA
EdotDataB = dataB["Edot"].to_numpy()/ENormB

LdotDataA = dataA["Ldot"].to_numpy()/LNormA
LdotDataB = dataB["Ldot"].to_numpy()/LNormB

In [82]:
EdotDataA_reshaped = EdotDataA.reshape(NY, NX, NZ).swapaxes(0, 1)
EdotDataB_reshaped = EdotDataB.reshape(NY, NX, NZ).swapaxes(0, 1)
LdotDataA_reshaped = LdotDataA.reshape(NY, NX, NZ).swapaxes(0, 1)
LdotDataB_reshaped = LdotDataB.reshape(NY, NX, NZ).swapaxes(0, 1)

In [83]:
EdotXYZ_A = TricubicSpline(xsamples, ysamples, zsamples, EdotDataA_reshaped)
EdotXYZ_B = TricubicSpline(xsamples, ysamples, zsamples, EdotDataB_reshaped)
LdotXYZ_A = TricubicSpline(xsamples, ysamples, zsamples, LdotDataA_reshaped)
LdotXYZ_B = TricubicSpline(xsamples, ysamples, zsamples, LdotDataB_reshaped)

In [84]:
def energy_flux(a, p, e):
    if valid_parameters_test(a, p, e) == False:
        return 0.*a
    
    Enorm = energy_flux_pn(p)
    if p < PCRITVALUE:
        x, y, z = transform_ape_to_xyzbar_regionA(a, p, e)
        return EdotXYZ_A(x, y, z)*Enorm
    else:
        x, y, z = transform_ape_to_xyzbar_regionB(a, p, e)
        return EdotXYZ_B(x, y, z)*Enorm
    
def angular_momentum_flux(a, p, e):
    if valid_parameters_test(a, p, e) == False:
        return 0.*a
    
    Lnorm = momentum_flux_pn(p)
    if p < PCRITVALUE:
        x, y, z = transform_ape_to_xyzbar_regionA(a, p, e)
        return LdotXYZ_A(x, y, z)*Lnorm
    else:
        x, y, z = transform_ape_to_xyzbar_regionB(a, p, e)
        return LdotXYZ_B(x, y, z)*Lnorm

## Calculate $\dot{E}$ and $\dot{L}$

In [86]:
print(energy_flux(0.99, 2, 0.3))
print(angular_momentum_flux(0.99, 2, 0.3))

0.05251088915848533
0.1772010200247519
